In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import os

#  Load  and prepare HW and NO_HW df

##  Load HW data

In [ ]:
# Load the parquet data from /Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/parquet
#data_dir = '/Users/yguo/DataSpellProjects/hw/data/parquet'
data_dir = '/Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/parquet'
file_name_hw = 'HW_1985_1994.parquet'
#join data_dir and file_name
hw_path = os.path.join(data_dir, file_name_hw)
df_hw = pd.read_parquet(hw_path)
print(df_hw.info())
df_hw

##  add hour, month and year to the df_hw


In [ ]:
# Ensure 'time' is of datetime type
df_hw.index = df_hw.index.set_levels([df_hw.index.levels[0], df_hw.index.levels[1], pd.to_datetime(df_hw.index.levels[2])])

# Extract hour, month and year from 'time'
df_hw['hour'] = df_hw.index.get_level_values('time').hour
df_hw['month'] = df_hw.index.get_level_values('time').month
df_hw['year'] = df_hw.index.get_level_values('time').year
df_hw

In [ ]:
# # Group by 'lat', 'lon', 'year', 'month', and 'hour', then calculate the mean of 'UHI' and 'UBWI'
# df_hw_avg = df_hw.groupby(['lat', 'lon', 'year', 'month', 'hour']).mean()
# df_hw_avg

##  Load NO_HW data

In [ ]:

file_name_no_hw = 'NO_HW_1985_1994.parquet'
#join data_dir and file_name
no_hw_path = os.path.join(data_dir, file_name_no_hw)
df_no_hw = pd.read_parquet(no_hw_path)
print(df_no_hw.info())
df_no_hw

In [ ]:
#todo: add UHI and NO_HW UHI to make sure they are the same as the oringal netcdf data


##  Validate there is not overlap between the HW and NO_HW data

In [ ]:
#the key for both df_hw and df_no_hw are lat, lon and time. please show python code that they don't overlap on those keys
# Convert the MultiIndex of both DataFrames to sets
# keys_hw = set(df_hw.index)
# keys_no_hw = set(df_no_hw.index)
# 
# # Check if the intersection of these sets is empty
# overlap = keys_hw & keys_no_hw
# 
# # If the intersection is empty, print that there is no overlap. Otherwise, print the overlapping keys.
# if not overlap:
#     print("There is no overlap between the keys of df_hw and df_no_hw.")
# else:
#     print("The following keys overlap between df_hw and df_no_hw:")
#     print(overlap)

In [ ]:

# group df_no_hw by lat, lon, year and hour of the day avaerage UHI and UBWI
df_no_hw.index = df_no_hw.index.set_levels(
    [df_no_hw.index.levels[0], df_no_hw.index.levels[1], pd.to_datetime(df_no_hw.index.levels[2])])
df_no_hw['hour'] = df_no_hw.index.get_level_values('time').hour
df_no_hw['year'] = df_no_hw.index.get_level_values('time').year
df_no_hw_avg = df_no_hw.groupby(['lat', 'lon', 'year', 'hour']).mean()
df_no_hw_avg

#  2: Calculate the difference between UHI in df_hw and df_no_hw

##  UHI HW - NO_HW ( HW hour data - NO_HW yearl average data for the hour) 
the df_no_hw_avg is the average value for a given hour of the day throughout the year.
In the 2018 Zhao paper they seem to just do average the whole 30 years. 
I want to substract the average UHI on the given hour for a given year from the hourly UHI data I have in df_hw, matching the year and hour between the two dataframes.


##  Step 2.1: Reset the index of df_hw and df_no_hw_avg (be careful on the increased memory usage)

In [ ]:
df_hw.info()
df_hw_reset = df_hw.reset_index()
df_hw_reset.info()
df_no_hw_avg.info()
df_no_hw_avg_reset = df_no_hw_avg.reset_index()
df_no_hw_avg_reset.info()



##   Step 2.2: Merge df_hw with df_no_hw_avg_reset

In [ ]:

merged_df = pd.merge(df_hw_reset, df_no_hw_avg_reset[['lat', 'lon', 'year', 'hour', 'UHI', 'UBWI']],
                     on=['lat', 'lon', 'year', 'hour'],
                     suffixes=('', '_avg'))

In [ ]:
merged_df.info()


##  Step 2.3: Subtract the average UHI from the hourly UHI and store in a new column

In [ ]:

merged_df['UHI_diff'] = merged_df['UHI'] - merged_df['UHI_avg']
merged_df['UBWI_diff'] = merged_df['UBWI'] - merged_df['UBWI_avg']
# Now, merged_df contains your original data along with the subtracted UHI values in 'UHI_diff'
merged_df  # To check the first few rows of the merged DataFrame

In [ ]:
merged_df[['UHI_diff', 'UBWI_diff']].describe()

In [ ]:
merged_df.info()

## Step 2.4: Validate the results by checking the UHI values for a specific location and time

In [ ]:

row_index = 198
print(merged_df.iloc[row_index].UHI_avg)
a_row = merged_df.iloc[row_index]
df_no_hw_avg_reset.loc[(df_no_hw_avg_reset['lat'] == a_row.lat) & (df_no_hw_avg_reset['lon'] == a_row.lon) & (
                df_no_hw_avg_reset['year'] == a_row.year) & (
                df_no_hw_avg_reset['hour'] == a_row.hour)].UHI


#  3: Averaged data for each local hour 

##  Step 3.1: Adjust to local hour

In [ ]:
import pandas as pd
import numpy as np

def convert_time_to_local_and_add_hour(df):
    """
    Converts the UTC timestamp in the DataFrame to local time based on longitude and adds a column for the local hour.
    
    Parameters:
    df (pd.DataFrame): DataFrame with columns for latitude ('lat'), longitude ('lon'), and UTC timestamp ('time')
    
    Returns:
    pd.DataFrame: DataFrame with additional columns ('local_time' and 'local_hour') for the timestamp adjusted to local time and the hour extracted from the local time
    """
    # Function to calculate timezone offset from longitude
    def calculate_timezone_offset(longitude):
        return np.floor(longitude / 15.0).astype(int)  # Approximate, not accounting for DST or specific timezone rules

    # Calculate timezone offsets for each row based on longitude
    offsets = calculate_timezone_offset(df['lon'].values)

    # Adjust timestamps by the offsets
    df['local_time'] = df['time'] + pd.to_timedelta(offsets, unit='h')

    # Extract the hour from the 'local_time' and create a new column
    df['local_hour'] = df['local_time'].dt.hour

    return df

# # Assuming 'df' is your original DataFrame
# # Make sure 'time' column is in datetime format
# df['time'] = pd.to_datetime(df['time'])

# # Convert UTC times to local times based on longitude and add local hour
# df = convert_time_to_local_and_add_hour(df)


In [ ]:
merged_df = convert_time_to_local_and_add_hour(merged_df)
merged_df


In [ ]:
merged_df.info()

##  Step 3.2 compute average based on local hour

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Group by 'lat', 'lon', and 'local_hour', then calculate the mean for 'UHI_diff'
# Ensure grouped data is sorted by 'lat' and 'lon' before pivoting
grouped_sorted = merged_df.groupby(['lat', 'lon', 'local_hour'])['UHI_diff'].mean().reset_index().sort_values(by=['lat', 'lon', 'local_hour'])

grouped_sorted


In [ ]:


# Convert the grouped and sorted data into an xarray Dataset for each local_hour
xarrays = {}
for hour in grouped_sorted['local_hour'].unique():
    # Filter the grouped DataFrame for the current local_hour
    hourly_data = grouped_sorted[grouped_sorted['local_hour'] == hour]
    
    # Ensure the data is sorted
    hourly_data = hourly_data.sort_values(by=['lat', 'lon'])
    
    # Pivot the data to create a 2D array of 'UHI_diff' values
    uhi_diff_grid = hourly_data.pivot(index='lat', columns='lon', values='UHI_diff').values

    # # Ensure there are no NaN values, which can cause issues with plotting
    # if np.isnan(uhi_diff_grid).any():
    #     print(f"Hour {hour} contains NaN values and will be skipped.")
    #     continue
    
    lat_unique = hourly_data['lat'].unique()
    lon_unique = hourly_data['lon'].unique()

    # Create an xarray DataArray with sorted coordinates
    xarrays[hour] = xr.DataArray(
        uhi_diff_grid,
        coords=[('lat', lat_unique), ('lon', lon_unique)],
        dims=['lat', 'lon'],
        name='UHI_diff'
    )




# 4: Plot and Explore

In [ ]:
hour_to_plot = 1
xr_data = xarrays.get(hour_to_plot)

if xr_data is not None:
    fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.set_title(f'UHI_diff Average for Local Hour: {hour_to_plot}')
    ax.coastlines()

    # Sort the DataArray if it's not already sorted
    xr_data_sorted = xr_data.sortby(['lat', 'lon'])
    
    im = xr_data_sorted.plot(ax=ax, add_colorbar=True, cmap='viridis', transform=ccrs.PlateCarree())
    im.colorbar.set_label('Average UHI_diff')
    plt.show()
else:
    print(f"No data available for local_hour {hour_to_plot}")

xr_data

In [ ]:
# import pandas as pd
# import numpy as np

# def convert_index_time_to_local(df):
#     """
#     Converts the UTC timestamp in the DataFrame's MultiIndex to local time based on longitude.
    
#     Parameters:
#     df (pd.DataFrame): DataFrame with a MultiIndex (latitude, longitude, UTC timestamp)
    
#     Returns:
#     pd.DataFrame: DataFrame with the timestamp in the MultiIndex adjusted to local time
#     """
#     # Function to calculate timezone offset from longitude
#     def calculate_timezone_offset(longitude):
#         return np.floor(longitude / 15.0).astype(int)  # Approximate, not accounting for DST or specific timezone rules

#     # Extract MultiIndex levels
#     latitudes, longitudes, timestamps = zip(*df.index)

#     # Convert to arrays for vectorized operations
#     longitudes = np.array(longitudes)
#     timestamps = pd.to_datetime(list(timestamps))

#     # Calculate timezone offsets
#     offsets = calculate_timezone_offset(longitudes)

#     # Adjust timestamps by the offsets
#     adjusted_timestamps = [timestamp + pd.Timedelta(hours=offset) for timestamp, offset in zip(timestamps, offsets)]

#     # Create a new MultiIndex with the adjusted timestamps
#     new_index = pd.MultiIndex.from_arrays([latitudes, longitudes, adjusted_timestamps], names=['Latitude', 'Longitude', 'LocalTime'])

#     # Assign the new MultiIndex to the DataFrame and return it
#     df.index = new_index
#     return df

# Usage example:
# new_df = convert_index_time_to_local(df)
# This will return a DataFrame with the index timestamps adjusted to local time based on longitude
# Note: ONLY CALL THIS FUNCTION ON THE DATAFRAME WITH THE MULTIINDEX (latitude, longitude, UTC timestamp), ALSO ONLY CALL AT THE FINAL STEP(AFTER ALL AGGREATION DONE!!!



In [ ]:
df_hw = convert_index_time_to_local(df_hw)
df_hw

In [ ]:
df_count = df_hw.groupby(['lat', 'lon']).size()
df_count

In [ ]:
import os
os.environ["PROJ_LIB"] = "/home/jguo/anaconda3/envs/I2000/share/proj"

import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

df_hw = pd.read_parquet('/Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/parquet/HW_1985_1994.parquet')
df_count = df_hw.groupby(['lat', 'lon']).size()
df_count_sorted = df_count.sort_values(ascending=False).head(50)
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)

# Add gridlines
gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

lats, lons = zip(*df_count_sorted.index)
ax.scatter(lons, lats, color='red', transform=ccrs.PlateCarree())

plt.show()

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df_count_sorted, geometry=gpd.points_from_xy(lons, lats))

# Load Natural Earth dataset with coastlines
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Plot the world (coastlines)
fig, ax = plt.subplots(figsize=(10, 5))
world.boundary.plot(ax=ax, color='black')

# Plot the GeoDataFrame
gdf.plot(marker='o', color='red', markersize=5, ax=ax)
plt.show()

In [ ]:
df_count_sorted.info()

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np

# Create a plot with coastlines
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

# Extract latitudes, longitudes, and data values from the MultiIndex Series
latitudes = df_count_sorted.index.get_level_values(0)
longitudes = df_count_sorted.index.get_level_values(1) % 360  # Adjust longitudes if they are in a 0-360 format
data_values = df_count_sorted.values

# Plot the data
scatter = plt.scatter(longitudes, latitudes, c=data_values, cmap='viridis', transform=ccrs.PlateCarree())

# Add gridlines and labels
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

# Add a colorbar
cbar = plt.colorbar(scatter, orientation='vertical', pad=0.02, aspect=50)
cbar.set_label('Data Value')

plt.show()




In [ ]:
# Convert the Series with a MultiIndex to an xarray DataArray
da = xr.DataArray(df_count_sorted, dims=['lat_lon'], name='data')
da['lat'] = ('lat_lon', df_count_sorted.index.get_level_values('lat'))
da['lon'] = ('lat_lon', df_count_sorted.index.get_level_values('lon'))
# Define the new grid (adjust the resolution as needed)
new_lat = np.linspace(da['lat'].min(), da['lat'].max(), num=100)
new_lon = np.linspace(da['lon'].min(), da['lon'].max(), num=100)

# Interpolate the DataArray onto the new grid
da_interp = da.interp(lat=new_lat, lon=new_lon)


In [ ]:
da.HW_Count.plot()


In [ ]:

df_no_hw = pd.read_parquet('/Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/parquet/NO_HW_1985_1994.parquet')

print(df_no_hw.info())  


In [ ]:
df_hw.index.levels[2]

In [ ]:
# Ensure 'time' is of datetime type
df_hw.index = df_hw.index.set_levels([df_hw.index.levels[0], df_hw.index.levels[1], pd.to_datetime(df_hw.index.levels[2])])

# Extract hour, month and year from 'time'
df_hw['hour'] = df_hw.index.get_level_values('time').hour
df_hw['month'] = df_hw.index.get_level_values('time').month
df_hw['year'] = df_hw.index.get_level_values('time').year

# Group by 'lat', 'lon', 'year', 'month', and 'hour', then calculate the mean of 'UHI' and 'UBWI'
df_avg = df_hw.groupby(['lat', 'lon', 'year', 'month', 'hour']).mean()
df_avg